In [ ]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

SyntaxError: invalid syntax (1642520217.py, line 3)

In [ ]:
# load processed data
df = pd.read_csv("cblol_training_dataV2.csv")

In [3]:
# champion names encoding
champ_cols = [col for col in df.columns if 'champion' in col]

In [4]:
# SINGLE encoder for all champions (e.g. whether Morgana is played in any position, it gets the same encoding like ID #23)
global_encoder = LabelEncoder()

In [5]:
# take all unique champions from all 10 columns to fit the encoder
all_champs = pd.concat([df[col] for col in champ_cols]).unique()
global_encoder.fit(all_champs)

LabelEncoder()

In [6]:
# apply encodinf to dataframe
for col in champ_cols:

    # handle unseen champions by mapping them to 'Unknown'
    df[col] = df[col].apply(lambda x: x if x in global_encoder.classes_ else 'Unknown')
    if 'Unknown' not in global_encoder.classes_:
        pass

    df[col] = global_encoder.transform(df[col])

In [7]:
# save the encoder for later use in app.py
joblib.dump(global_encoder, "champion_label_encoder.pkl")
print("Champion Label Encoder saved.")


Champion Label Encoder saved.


In [8]:
# defining feaatures (x) and target (y) && dropping metadata columns not needed
features_to_drop = ['gameid', 'date', 'Blue_Team', 'Red_Team', 'blue_win_label', 'patch', 'patch_blue', 'patch_red', 'teamname_red', 'teamname_blue']

In [9]:
# select only numeric features and encoded champion columns
valid_features = [col for col in df.columns if col not in features_to_drop]

X = df[valid_features]
y = df['blue_win_label']

In [10]:
# train-test split: 80-20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [11]:
# init the brain (XGBoost)
model = XGBClassifier(
    n_estimators=200,      # Number of "trees" in the forest
    learning_rate=0.05,    # How fast it learns (lower is more careful)
    max_depth=4,           # How complex each tree is
    random_state=42
)

In [12]:
print(f"Training on {len(X_train)} matches...")
model.fit(X_train, y_train)

Training on 608 matches...


,"objective objective: str | xgboost.sklearn._SklObjWProto | typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]] | NoneSpecify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: float | typing.List[float] | NoneThe initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.List[xgboost.callback.TrainingCallback] | NoneList of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: float | NoneSubsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: float | NoneSubsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: float | NoneSubsample ratio of columns when constructing each tree.,None
,"device device: str | None.. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: int | None.. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: str | typing.List[str | typing.Callable] | typing.Callable | None.. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes from sklearn.metrics import mean_absolute_error X, y = load_diabetes(return_X_y=True) reg = xgb.XGBRegressor( tree_meth

In [ ]:
# predctions
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"\n🏆 Model Accuracy: {accuracy:.2%}")
print("\nDetailed Re    port:")
print(classification_report(y_test, predictions))


🏆 Model Accuracy: 56.86%

Detailed Report:
              precision    recall  f1-score   support

           0       0.47      0.47      0.47        62
           1       0.64      0.64      0.64        91

    accuracy                           0.57       153
   macro avg       0.55      0.55      0.55       153
weighted avg       0.57      0.57      0.57       153

